In [1]:
import tensorflow_mri as tfmr
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.backend as K
import random
import matplotlib.pyplot as plt
import neptune.new as neptune
import os
from scipy import ndimage
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import glob
from neptune.new.integrations.tensorflow_keras import NeptuneCallback
import matplotlib.animation as animation
import ipynbname
nb_fname = ipynbname.name()
from losses import *
from scipy import stats
from unet3plus import *
from pathlib import Path
import pydicom
from frechetdist import frdist
import matplotlib.patches as mpatches
import seaborn as sns
import pandas as pd
from scipy.interpolate import CubicSpline
from tqdm import tqdm
from scipy.ndimage import zoom 

2024-10-03 16:51:03.612601: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
vessels_dict = {'lpa':0,'rpa':1,'ao':2,'svc':3,'ivc':4}
swap_patients = []#['COL-RAMDOM-1_2022','PIT-PHIZAC-1_2018','BCH-CARJOS-1_2021','PIT-STATER-1_2023']
# double_aorta_patients = ['BCH-BOYPAU-1_2016','BCH-SNOCHA-1_2024','BCH-VENJAM-1_2023','PIT-WINNIC-1_2022','UKY-BROEZE-1_2022','LCH-CORJAS-1_2014']
double_aorta_patients = ['BCH-BOYPAU-1_2016','MTS-RAMLAU-1_2021','IND-PAULIS-1_2021','MOT-THOPET-1_2019','OUH-COXCAL-1_2021','YAL-JAVELI-1_2021','YAL-PLETAY-1_2020']
# ['BCH-SNOCHA-1_2024','BCH-VENJAM-1_2023','PIT-WINNIC-1_2022']
error_patients = ['CHO-JANHAN-1_2019']
new_patients = ["BCH-ALEOLI-1_2022","BCH-MAFSAM-1_2024","BCH-VARADA-1_2017","BCH-YOULOG-1_2024","CHP-HEREDG-1_2021","CHP-NGUSOP-1_2018"]#['BCH-BOYPAU-1_2016','MTS-RAMLAU-1_2021','IND-PAULIS-1_2021','MOT-THOPET-1_2019','OUH-COXCAL-1_2021','YAL-JAVELI-1_2021','YAL-PLETAY-1_2020']


In [3]:
data_path = '../data/clean'
patients =sorted(np.unique(['_'.join(pat.split('_')[:2]).split('/')[-1].replace('.npy','') for pat in glob.glob(f'../data/clean/*')]))
print(len(patients))
desired_frames = 32

260


In [4]:
for patient in tqdm(['YAL-LIVMIC-1_2023']):
#     try:
    if not os.path.exists(f'../data/clean_128_32/{patient}_ivc.npy') or 1:
        for vessel in vessels_dict.keys(): 

            vessel_mag_image, vessel_phase_image, vessel_masks = np.load(f'{data_path}/{patient}_{vessel}.npy', allow_pickle = True)
            frames = vessel_mag_image.shape[-1]
            ratio = desired_frames/frames

            mag_im = zoom(vessel_mag_image, (0.5,0.5,ratio))
            phase_im = zoom(vessel_phase_image, (0.5,0.5,ratio))
            mask = zoom(vessel_masks, (0.5,0.5,ratio))

            mask[mask> 0.5] = 1
            mask[mask<= 0.5] = 0


            np.save(f'../data/clean_128_32/{patient}_{vessel}.npy', [mag_im, phase_im, mask])
#     except Exception as e:
#         print(e, patient)

100%|██████████| 1/1 [00:02<00:00,  2.71s/it]


In [5]:
# all_frames = []
# for patient in tqdm(patients):
#     vessel_mag_image, vessel_phase_image, vessel_masks = np.load(f'{data_path}/{patient}_lpa.npy', allow_pickle = True)
#     frames = vessel_mag_image.shape[-1]
#     all_frames.append(frames)

In [6]:
np.median(all_frames)

NameError: name 'all_frames' is not defined

In [ ]:
np.mean(all_frames)

In [ ]:
plt.hist(all_frames)

In [ ]:
df = pd.DataFrame()

In [ ]:
df['frames'] = all_frames

In [ ]:
df['hosp'] = [pat[:3] for pat in patients]

In [ ]:
sns.barplot(data = df, y = 'frames', x = 'hosp')